# Compare fit results to published values

This notebook compares fit results from SNCosmo with fit results published by SDSS and DES.

In [ ]:
import numpy as np
import sncosmo
from astropy.table import Table, join
from matplotlib import pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Span, HoverTool, ColumnDataSource
from bokeh.layouts import gridplot
from tqdm import tqdm

import sys; sys.path.insert(0, '../')
from data_access import sdss_data, des_data

In [ ]:
# Set up bokeh plotting
output_notebook()
_basic_tools = "save,pan,box_zoom,reset,wheel_zoom".split(',')


## Plotting functions

Since we will be creating the same plots for multiple data sets, we create some generic plotting functions. We create a static version of the plot zoomed in to show detail, along with an interactive version that defaults to show all available data. Note that we assume the colum names of the data tables being plotted.

In [ ]:
def create_static_figure(fit_data, survey_name):
    """Return a matplotlib figure comparing x0, x1, c, and chisq
    of the published and SNCosmo fit values.
    
    Args:
        fit_data  (Table): A table of data to plot
        survey_name (str): Name of the survey being plotted
        
    Returns:
         A matplotlib figure object
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    line = (-20, 2000)

    for axis, value in zip(axes, ['x0', 'x1', 'c']):
        axis.plot(line, line, linestyle='--', color='grey')
        axis.errorbar(fit_data['{}_{}'.format(value, survey_name)], 
                      fit_data['{}_sncosmo'.format(value)], 
                      xerr=fit_data['{}_{}_err'.format(value, survey_name)],
                      yerr=fit_data['{}_err_sncosmo'.format(value)], 
                      linestyle='',
                      alpha=.3)

    
    reduced_chisq_survey = fit_data['chisq_{}'.format(survey_name)] / fit_data['ndof_{}'.format(survey_name)]
    reduced_chisq_sncosmo = fit_data['chisq_sncosmo'] / fit_data['ndof_sncosmo']
    axes[3].plot(line, line, linestyle='--', color='grey', alpha=.8)
    axes[3].scatter(fit_data['chisq_{}'.format(survey_name)], 
                    fit_data['chisq_sncosmo'], 
                    alpha=.3, s=10)
    
    axes[4].plot(line, line, linestyle='--', color='grey', alpha=.8)
    axes[4].scatter(reduced_chisq_survey, 
                    reduced_chisq_sncosmo, 
                    alpha=.3, s=10)

    axes[0].set_title('x0')
    axes[0].set_ylabel('SNCosmo Fit ugriz')

    axes[1].set_title('x1')

    axes[2].set_title('c')
    axes[2].set_xlabel('Published Value')
    axes[2].set_ylabel('SNCosmo Fit ugriz')

    axes[3].set_xlabel('Published Value')
    axes[3].set_title('chisq')
    
    axes[4].set_xlabel('Published Value')
    axes[4].set_title('chisq_norm')
    
    return axes


def create_interactive_figure(fit_data, survey_name):
    """Return a bokeh figure comparing x0, x1, c, and chisq
    of the published and SNCosmo fit values.
    
    Args:
        fit_data  (Table): A table of data to plot
        survey_name (str): Name of the survey being plotted
        
    Returns:
         A bokeh figure object
    """
    
    data_dict = {col.name: np.array(col) for col in fit_data.itercols()}
    data_dict['chisq_norm_sncosmo'] = data_dict['chisq_sncosmo'] / data_dict['ndof_sncosmo']
    data_dict['chisq_norm_{}'.format(survey_name)] = data_dict['chisq_{}'.format(survey_name)] / data_dict['ndof_{}'.format(survey_name)]
    source = ColumnDataSource(data=data_dict)

    hover = HoverTool(tooltips=[
            ("target", "@cid"),
            ("class", "@class"),
            ("z", "@z"),
            ('z_fit', '@fit_z'),
            ('chi2', '@chi'),
            ('dof', '@dof'),
        ])

    figures = []
    for value in ('x0', 'x1', 'c', 'chisq_norm'):
        fig = figure(tools=_basic_tools + [hover, 'box_select', 'lasso_select'], title=value)
        fig.circle('{}_{}'.format(value, survey_name), 
                   '{}_sncosmo'.format(value), 
                   source=source, size=4, alpha=.5)
        
        fig.xaxis.axis_label = 'Published Value'
        fig.yaxis.axis_label = 'SNCosmo Fit ugriz'
        figures.append(fig)
    
    return gridplot(figures, ncols=2, plot_width=350, plot_height=350)


## SDSS

We create a table to store both the published and SNCosmo fit results for SDSS. Then we plot correlations between both data sets for the fit parameters x0, x1 (stretch), and c (color) along with normalized chi-squared.

In [ ]:
# Get SDSS published data
sdss_published = sdss_data.master_table[
    'CID',
    'x0SALT2zspec', 
    'x0errSALT2zspec', 
    'x1SALT2zspec', 
    'x1errSALT2zspec', 
    'cSALT2zspec', 
    'cerrSALT2zspec',
    'chi2SALT2zspec',
    'ndofSALT2zspec'
]

# Rename columns for consistancy
sdss_published.rename_column('CID', 'cid')
sdss_published.rename_column('x0SALT2zspec', 'x0_sdss')
sdss_published.rename_column('x0errSALT2zspec', 'x0_sdss_err')
sdss_published.rename_column('x1SALT2zspec', 'x1_sdss')
sdss_published.rename_column('x1errSALT2zspec', 'x1_sdss_err')
sdss_published.rename_column('cSALT2zspec', 'c_sdss')
sdss_published.rename_column('cerrSALT2zspec', 'c_sdss_err')
sdss_published.rename_column('chi2SALT2zspec', 'chisq_sdss')
sdss_published.rename_column('ndofSALT2zspec', 'ndof_sdss')

# Get SNCosmo fit data
sdss_sncosmo = Table.read('../sncosmo/sdss_results/snia_ugriz.csv')
sdss_sncosmo.rename_column('x0', 'x0_sncosmo')
sdss_sncosmo.rename_column('x0_err', 'x0_err_sncosmo')
sdss_sncosmo.rename_column('x1', 'x1_sncosmo')
sdss_sncosmo.rename_column('x1_err', 'x1_err_sncosmo')
sdss_sncosmo.rename_column('c', 'c_sncosmo')
sdss_sncosmo.rename_column('c_err', 'c_err_sncosmo')
sdss_sncosmo.rename_column('chi', 'chisq_sncosmo')
sdss_sncosmo.rename_column('dof', 'ndof_sncosmo')

# Combine tables and keep only SN with published fits
combined_sdss = join(sdss_sncosmo, sdss_published)
combined_sdss = combined_sdss[~combined_sdss['x0_sdss'].mask]   
combined_sdss[:5].show_in_notebook()


In [ ]:
# Consider only type 1a objects
fit_data = combined_sdss[combined_sdss['class'] == 'SNIa']
plot_axes = create_static_figure(fit_data, 'sdss')

plot_axes[0].set_xlim(0, .003)
plot_axes[0].set_ylim(0, .003)
plot_axes[1].set_xlim(-10, 10)
plot_axes[1].set_ylim(-10, 10)
plot_axes[2].set_xlim(-1, 1)
plot_axes[2].set_ylim(-1, 1)
plot_axes[3].set_xlim(0, 200)
plot_axes[3].set_ylim(0, 1000)
plot_axes[4].set_xlim(0, 10)
plot_axes[4].set_ylim(0, 20)

plt.show()

In [ ]:
plot_grid = create_interactive_figure(combined_sdss, 'sdss')
show(plot_grid)


## DES

As before, we create a table to store both the published and SNCosmo fit results and plot the correlation.

In [ ]:
# Get DES published data
des_published = des_data.master_table[
    'CID',
    'x0',
    'x0ERR',
    'x1',
    'x1ERR',
    'c',
    'cERR',
    'FITCHI2',
    'NDOF'
]

des_published.rename_column('CID', 'cid')
des_published.rename_column('x0', 'x0_des')
des_published.rename_column('x0ERR', 'x0_des_err')
des_published.rename_column('x1', 'x1_des')
des_published.rename_column('x1ERR', 'x1_des_err')
des_published.rename_column('c', 'c_des')
des_published.rename_column('cERR', 'c_des_err')
des_published.rename_column('FITCHI2', 'chisq_des')
des_published.rename_column('NDOF', 'ndof_des')

# Get SNCosmo fit data
des_sncosmo = Table.read('../sncosmo/des_results/snia_ugriz.csv')
des_sncosmo.rename_column('x0', 'x0_sncosmo')
des_sncosmo.rename_column('x0_err', 'x0_err_sncosmo')
des_sncosmo.rename_column('x1', 'x1_sncosmo')
des_sncosmo.rename_column('x1_err', 'x1_err_sncosmo')
des_sncosmo.rename_column('c', 'c_sncosmo')
des_sncosmo.rename_column('c_err', 'c_err_sncosmo')
des_sncosmo.rename_column('chi', 'chisq_sncosmo')
des_sncosmo.rename_column('dof', 'ndof_sncosmo')
des_sncosmo['cid'] = [str(x) for x in des_sncosmo['cid']]

# Combine tables and keep only SN with published fits
combined_des = join(des_sncosmo, des_published)
combined_des[:5].show_in_notebook()


In [ ]:
plot_axes = create_static_figure(combined_des, 'des')

plot_axes[0].set_xlim(0, 5e-4)
plot_axes[0].set_ylim(0, 5e-4)
plot_axes[1].set_xlim(-5.5, 5.5)
plot_axes[1].set_ylim(-5.5, 5.5)
plot_axes[2].set_xlim(-1, 1)
plot_axes[2].set_ylim(-1, 1)
plot_axes[3].set_xlim(0, 70)
plot_axes[3].set_ylim(0, 200)
plot_axes[4].set_xlim(0, 2)
plot_axes[4].set_ylim(0, 8)

plt.show()


In [ ]:
plot_grid = create_interactive_figure(combined_des, 'des')
show(plot_grid)


## Exploring disagreements bewteen DES results

Lets inspect a few light curves where the chi-squared values differ significantly between the published and SNCsomo fit results. We can then fit the lightcurve in SNCosmo with and without specifying the published value of each parameter and compare the results.


In [ ]:
test_ids = (1330031, 1330426)

for test_id in test_ids:
    input_table = des_data.get_input_for_id(test_id)

    # Get published values
    published_values = des_published[des_published['cid'] == str(test_id)]
    x0 = published_values['x0_des'][0]
    x1 = published_values['x1_des'][0]
    c = published_values['c_des'][0]
    chisq_norm = published_values['chisq_des'][0] / published_values['ndof_des'][0]

    print('Published Values for {}:'.format(test_id))
    print('x0: ', x0)
    print('x1: ', x1)
    print('c: ', c)
    print('chisq_norm: ', chisq_norm)

    print('\n\nFitting for all terms except z:')
    model = sncosmo.Model(source='salt2')
    model.set(z=input_table.meta['redshift'])

    result, fitted_model = sncosmo.fit_lc(input_table, model, ['t0', 'x0', 'x1', 'c'], bounds=None)
    result['chi2_norm'] = (result.chisq / result.ndof)
    sncosmo.plot_lc(input_table, model=fitted_model, errors=result.errors)
    plt.show()
    print(result)

    print('\n\nFitting with fixed, published values:')
    model.set(z=input_table.meta['redshift'], x0=x0, x1=x1, c=c)
    result, fitted_model = sncosmo.fit_lc(input_table, model, ['t0'], bounds=None)
    result['chi2_norm'] = (result.chisq / result.ndof)
    sncosmo.plot_lc(input_table, model=fitted_model, errors=result.errors)
    plt.show()
    print(result)
    print('\n\n\n')

Next we run fits for all DES targets fitting only for `t0`. All other parameters are set to published values.

In [ ]:
col_names = ['cid', 'z_sncosmo','t0_sncosmo', 
             'x0_sncosmo', 'x1_sncosmo', 'c_sncosmo', 
             'chisq_sncosmo', 'ndof_sncosmo']

test_fit_results = Table(names=col_names,
    dtype = ['S30'] + [float for _ in col_names[:-1]],
)

for input_table in des_data.iter_sncosmo_input(verbose=True):
    cid = str(input_table.meta['cid'])
    published_values = des_published[des_published['cid'] == cid]
    x0 = published_values['x0_des']
    x1 = published_values['x1_des']
    c = published_values['c_des']
    
    model = sncosmo.Model(source='salt2')
    model.set(z=input_table.meta['redshift'], x0=x0, x1=x1, c=c)
    
    try:
        result, fitted_model = sncosmo.fit_lc(
            input_table, model, ['t0'], bounds=None)

        new_row = [cid]
        new_row.extend(result.parameters)
        new_row.append(result.chisq)
        new_row.append(result.ndof)
        test_fit_results.add_row(new_row)
        
    except:
        tqdm.write('Failed: ' + cid)



In [ ]:
plot_data = join(test_fit_results, des_published)
plot_grid = create_interactive_figure(plot_data, 'des')
show(plot_grid)
